# Modern Data Science
**(Module 07: Natural Language Processing)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, but NOT allowed to change or distribute this package.

Prepared by and for 
**Student Members** |
2006-2019 [TULIP Lab](http://www.tulip.org.au)

---

# Session E - Glove

## Contents

1 [Import Package and corpus](#Import)

2 [vocab_count: Calculating Word Statistics](#Calculating)

3 [cooccur: Calculate Co-occurrence](#cooccur)

4 [shuffle: Reorganization of co-occurrence results](#shuffle)

5 [Training](#Training)

6 [Similarity calculation](#Similarity)


<a id = "Import"></a>
##  <span style="color:#0b486b">Import Package and corpus</span>


In [ ]:
from collections import Counter
import numpy as np
from scipy import sparse
import itertools
from random import shuffle
from math import log

In [ ]:
corpus = ('''Lionel Messi born 24 June 1987 is an Argentine professional footballer who plays as a forward for 
Spanish club Barcelona and the Argentine national team. Often considered the best player in the world and regarded
by many as the greatest of all time, Messi has a record-tying five Ballon d'Or awards,four of which he won 
consecutively, and a record five European Golden Shoes. He has spent his entire professional career with Barcelona, 
where he has won 32 trophies, including nine La Liga titles, four UEFA Champions League titles, and six Copas 
del Rey. Both a prolific goalscorer and a creative playmaker, Messi holds the records for most official goals 
scored in La Liga (383), a La Liga season (50), a club football season in Europe (73), a calendar year (91), 
El Clásico (26), as well as those for most assists in La Liga (149) and the Copa América (11). He has scored over 
600 senior career goals for club and country.

Born and raised in central Argentina, Messi was diagnosed with a growth hormone deficiency as a child. At age 13, 
he relocated to Spain to join Barcelona, who agreed to pay for his medical treatment. After a fast progression 
through Barcelona's youth academy, Messi made his competitive debut aged 17 in October 2004. Despite being 
injury-prone during his early career, he established himself as an integral player for the club within the next 
three years, finishing 2007 as a finalist for both the Ballon d'Or and FIFA World Player of the Year award, a feat 
he repeated the following year. His first uninterrupted campaign came in the 2008–09 season, during which he helped 
Barcelona achieve the first treble in Spanish football. At 22 years old, Messi won the Ballon d'Or and FIFA World 
Player of the Year award by record voting margins.

Three successful seasons followed, with Messi winning three consecutive FIFA Ballons d'Or, including an 
unprecedented fourth. His best campaign statistically to date was the 2011–12 season, in which he set the La Liga 
and European records for most goals scored in a single season, while establishing himself as Barcelona's all-time 
top scorer in official competitions in March 2012. The following two seasons, Messi finished twice second for the 
Ballon d'Or behind Cristiano Ronaldo, his perceived career rival. Messi regained his best form during the 2014–15 
campaign, breaking the all-time goalscoring records in both La Liga and the Champions League in November 2014,
[note 3] and led Barcelona to a historic second treble.

An Argentine international, Messi is his country's all-time leading goalscorer. At youth level, he won the 2005 
FIFA World Youth Championship, finishing the tournament with both the Golden Ball and Golden Shoe, and an Olympic 
gold medal at the 2008 Summer Olympics. His style of play as a diminutive, left-footed dribbler drew comparisons 
with compatriot Diego Maradona, who declared the teenager his successor. After making his senior debut in August 
2005, Messi became the youngest Argentine to play and score in a FIFA World Cup during the 2006 edition, and reached
the final of the 2007 Copa América, where he was named young player of the tournament. As the squad's captain from 
August 2011, he led Argentina to three consecutive finals: the 2014 World Cup, for which he won the Golden Ball, 
and the 2015 and 2016 Copas América. After announcing his international retirement in 2016, he reversed his 
decision and led his country to qualification for the 2018 World Cup.''').split("\n")

<a id = "Calculating"></a>
## <span style="color:#0b486b">vocab_count: Calculating Word Statistics</span>

In [ ]:
def build_vocab(corpus):
    
    vocab = Counter()
    for line in corpus:
        tokens = line.strip().split()
        vocab.update(tokens)

    return {word: (i, freq) for i, (word, freq) in enumerate(vocab.items())}

In [ ]:
vocab = build_vocab(corpus)
vocab

<a id = "cooccur"></a>
## <span style="color:#0b486b">cooccur: Calculate Co-occurrence</span>

In [ ]:
def build_cooccur(vocab, corpus, window_size=10, min_count=None):

    vocab_size = len(vocab)
    id2word = dict((i, word) for word, (i, _) in vocab.items())

    cooccurrences = sparse.lil_matrix((vocab_size, vocab_size), dtype=np.float64)

    for i, line in enumerate(corpus):
        tokens = line.strip().split()
        token_ids = [vocab[word][0] for word in tokens]

        for center_i, center_id in enumerate(token_ids):
            context_ids = token_ids[max(0, center_i - window_size) : center_i]
            contexts_len = len(context_ids)

            for left_i, left_id in enumerate(context_ids):
                # Distance from center word
                distance = contexts_len - left_i

                # Weight by inverse of distance between words
                increment = 1.0 / float(distance)

                # Build co-occurrence matrix symmetrically (pretend we
                # are calculating right contexts as well)
                cooccurrences[center_id, left_id] += increment
                cooccurrences[left_id, center_id] += increment

    for i, (row, data) in enumerate(zip(cooccurrences.rows,
                                                   cooccurrences.data)):
        if min_count is not None and vocab[id2word[i]][1] < min_count:
            continue
        for data_idx, j in enumerate(row):
            if min_count is not None and vocab[id2word[j]][1] < min_count:
                continue
            
            yield i, j, data[data_idx]

In [ ]:
cooccurrences = build_cooccur(vocab, corpus, window_size=10)

In [ ]:
cooccurrences = list(cooccurrences)


<a id = "shuffle"></a>
## <span style="color:#0b486b">shuffle: Reorganization of co-occurrence results</span>

In [ ]:
def run_iter(vocab, data, learning_rate=0.05, x_max=100, alpha=0.75):

    global_cost = 0

    shuffle(data)

    for (v_main, v_context, b_main, b_context, gradsq_W_main, gradsq_W_context,
         gradsq_b_main, gradsq_b_context, cooccurrence) in data:

        weight = (cooccurrence / x_max) ** alpha if cooccurrence < x_max else 1

        cost_inner = (v_main.dot(v_context)
                      + b_main[0] + b_context[0]
                      - log(cooccurrence))

        cost = weight * (cost_inner ** 2)

        global_cost += 0.5 * cost

        grad_main = weight * cost_inner * v_context
        grad_context = weight * cost_inner * v_main

        grad_bias_main = weight * cost_inner
        grad_bias_context = weight * cost_inner

        v_main -= (learning_rate * grad_main / np.sqrt(gradsq_W_main))
        v_context -= (learning_rate * grad_context / np.sqrt(gradsq_W_context))

        b_main -= (learning_rate * grad_bias_main / np.sqrt(gradsq_b_main))
        b_context -= (learning_rate * grad_bias_context / np.sqrt(
                gradsq_b_context))

        gradsq_W_main += np.square(grad_main)
        gradsq_W_context += np.square(grad_context)
        gradsq_b_main += grad_bias_main ** 2
        gradsq_b_context += grad_bias_context ** 2

    return global_cost

<a id = "Training"></a>
## <span style="color:#0b486b">Training</span>

In [ ]:
def train_glove(vocab, cooccurrences, iter_callback=None, vector_size=100,
                iterations=25, **kwargs):

    vocab_size = len(vocab)
    
    W = (np.random.rand(vocab_size * 2, vector_size) - 0.5) / float(vector_size + 1)
    
    biases = (np.random.rand(vocab_size * 2) - 0.5) / float(vector_size + 1)
    
    gradient_squared = np.ones((vocab_size * 2, vector_size),
                               dtype=np.float64)

    gradient_squared_biases = np.ones(vocab_size * 2, dtype=np.float64)

    data = [(W[i_main], W[i_context + vocab_size],
             biases[i_main : i_main + 1],
             biases[i_context + vocab_size : i_context + vocab_size + 1],
             gradient_squared[i_main], gradient_squared[i_context + vocab_size],
             gradient_squared_biases[i_main : i_main + 1],
             gradient_squared_biases[i_context + vocab_size
                                     : i_context + vocab_size + 1],
             cooccurrence)
            for i_main, i_context, cooccurrence in cooccurrences]

    for i in range(iterations):
        
        cost = run_iter(vocab, data, **kwargs)

        if iter_callback is not None:
            iter_callback(W)

    return W

In [ ]:
W = train_glove(vocab, cooccurrences, iter_callback=None, vector_size=100, iterations=25)

In [ ]:
W

In [ ]:
word_id = vocab['Messi'][0]
word_id

In [ ]:
Messi = W[word_id]
Messi

In [ ]:
Argentina = W[vocab['Argentina'][0]]

<a id = "Similarity"></a>

## <span style="color:#0b486b">Similarity calculation</span>

In [ ]:
from scipy.spatial import distance
distance.cosine(Messi, Argentina)

In [ ]:
from scipy.spatial import distance
distance.cosine(W[vocab['Argentina'][0]], W[vocab['Spain'][0]])

In [ ]:
from scipy.spatial import distance
distance.cosine(W[vocab['Lionel'][0]], W[vocab['FIFA'][0]])

In [ ]:
distance.cosine(W[vocab['born'][0]], W[vocab['March'][0]])